## Imports

In [1]:
import gmpy2


## Read info from files

In [2]:
se = 'SE_12.txt'
mitm = 'MitM_12.txt'

se_C_data = []
se_N_data = []
mitm_data = []

with open(se, 'r') as f:
    lines = f.readlines()

    for index in range(0, len(lines), 2):
        se_C_data.append(int(lines[index].split('=')[1].strip(), base=16))
        se_N_data.append(int(lines[index + 1].split('=')[1].strip(), base=16))

with open(mitm, 'r') as f:
    lines = f.readlines()
    mitm_data.append(int(lines[0].split('=')[1].strip(), base=16))
    mitm_data.append(int(lines[1].split('=')[1].strip(), base=16))


## Chinese Remainder Theorem Attack

In [3]:
def gcdExtended(a, b):
    if a == 0 :
        return b, 0, 1

    gcd, u1, v1 = gcdExtended(b % a, a)

    u = v1 - (b // a) * u1
    v = u1

    return gcd, u, v

def CRT(arr_C, arr_N):
	if len(arr_C) != len(arr_N):
		return None

	main_modulo = 1
	for elem in arr_N:
		main_modulo *= elem

	modules = [main_modulo // elem for elem in arr_N]
	answer = 0

	for index in range(len(arr_C)):
		gcd, u, v = gcdExtended(arr_N[index], modules[index])

		answer += v * modules[index] * arr_C[index]

	return answer % main_modulo


In [4]:
E = 5
big_C = CRT(se_C_data, se_N_data)

message = gmpy2.iroot(big_C, E)

print(message[0] ** E == big_C)


True


## Meet in the Middle Attack

In [ ]:
def meet_in_the_middle(c, e, n, l):
	t_array = []

	current_range = [*range(0, 1 << (l >> 1))]

	for i in current_range:
		t_array.append(gmpy2.powmod(i + 1, e, n))

	for i in current_range:
		c_s = gmpy2.f_mod(gmpy2.mul(c, gmpy2.invert(t_array[i], n)), n)

		for j in current_range:
			if c_s == t_array[j]:
				return (i + 1, j + 1)

	return None


In [ ]:
c, n = mitm_data
e = 65537
l = 20

result = meet_in_the_middle(c, e, n, l)

if result != None:
	t, s = result
	print(gmpy2.powmod(t * s, e, n) == c)
else:
	print(':(')
